## In this tutorial you will learn how to transform obj files to precomputed format and host them on local server to be displayed by local neuroglancer instance. This can be used to generated precomputed versions of large amounts of light level data, to be co-visualised in the neuroglancer framework.

In [1]:
import os
import navis

In [2]:
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.volumes import uploadmultiresmeshes, uploadsingleresmeshes
from pyroglancer.ngviewer import openviewer, closeviewer
from pyroglancer.ngspaces import create_ngspace
from pyroglancer.createconfig import createconfig
from pyroglancer.layers import create_nglayer, setlayerproperty, get_ngserver, add_hostedlayer

In [3]:
#location to put the precomputed dataset at
hostdirectory = '/Users/sri/Documents/dataset/lightleveldata/'

In [4]:
#location where the obj files are located
meshfoldername = "data/"

## Step 1: Transform obj to precomputed format

### Here we create list of of obj meshes, arbitrary numerical mesh ids and mesh labels that we will take from file names.

In [5]:
meshes = []
meshes_id = []
meshes_nm = []
for e,f in enumerate(os.listdir(meshfoldername)):
    nrnmesh = navis.MeshNeuron(os.path.join(meshfoldername, f))
    nrnmesh.segid = 100 + e # we add some arbitrary id to our meshes
    nrnmesh.vertices = nrnmesh.vertices*1000 # scale vertices to proper unit if necessary
    meshes.append(nrnmesh)
    meshes_id.append(100 + e)
    meshes_nm.append(f.split(".")[0])

In [6]:
mesh_type = "singleresolution"

In [7]:
layer_name = 'obj_meshes'

In [8]:
if mesh_type == "singleresolution":
    uploadsingleresmeshes(meshes, meshes_id, meshes_nm, hostdirectory, layer_name)
elif mesh_type == "multiresolution":
    uploadmultiresmeshes(meshes, meshes_id, meshes_nm, hostdirectory, layer_name)
else:
    raise Exception("Unrecognised resolution type")

mesh/100
Seg id is: 100
Full filepath: /Users/sri/Documents/dataset/lightleveldata/precomputed/obj_meshes/mesh/100
mesh/101
Seg id is: 101
Full filepath: /Users/sri/Documents/dataset/lightleveldata/precomputed/obj_meshes/mesh/101
creating: /Users/sri/Documents/dataset/lightleveldata/precomputed/obj_meshes/mesh/segment_properties
creating: /Users/sri/Documents/dataset/lightleveldata/precomputed/obj_meshes/mesh/segment_names


## Step 2: Co-visualise

In [9]:
startdataserver(directory=hostdirectory)

Serving data from:  /Users/sri/Documents/dataset/lightleveldata/
Serving directory at http://127.0.0.1:8000


### Here you specify config YAML for your scene, so it contains other elements like EM data, neuropil meshes etc.

In [10]:
layer_kws = {'ngspace': 'MANC',
            "configfileloc": "/Users/sri/.pyroglancer/config.yml"}
create_ngspace(layer_kws)

config file loc is at: /Users/sri/.pyroglancer/config.yml
using custom location at:  /Users/sri/.pyroglancer/config.yml
Neuroglancer viewer created at:  http://127.0.0.1:52158/v/b0c34e7122e0c07b4fc21c700142c7ee0e194497/
config file loc is at: None
using default location at:  ~/.pyroglancer/config.yml
Dimensions are in : MANC
Layer created:  image
config file loc is at: None
using default location at:  ~/.pyroglancer/config.yml
Using layout : xy-3d
config file loc is at: None
using default location at:  ~/.pyroglancer/config.yml
Dimensions are in : MANC
Layer created:  segmentmesh
config file loc is at: None
using default location at:  ~/.pyroglancer/config.yml
Using layout : xy-3d
config file loc is at: None
using default location at:  ~/.pyroglancer/config.yml
Dimensions are in : MANC
Layer created:  segmentation
config file loc is at: None
using default location at:  ~/.pyroglancer/config.yml
Using layout : xy-3d


In [11]:
layer_serverdir, layer_host = get_ngserver()

### We add our locally hosted layer now..

In [12]:
tmpviewer = add_hostedlayer(layer_kws = {'type': 'volumes',
                                         'name': layer_name,
                                         'host': layer_host,
                                         'alpha': 0.5})

127.0.0.1 - - [28/Jul/2021 17:48:56] "GET /precomputed/obj_meshes/mesh/info HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 17:48:56] "GET /precomputed/obj_meshes/mesh/segment_properties/info HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 17:49:17] "GET /precomputed/obj_meshes/mesh/100:0 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 17:49:17] "GET /precomputed/obj_meshes/mesh/101:0 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 17:49:17] "GET /precomputed/obj_meshes/mesh/100 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 17:49:17] "GET /precomputed/obj_meshes/mesh/101 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 17:50:38] "GET /precomputed/obj_meshes/mesh/info HTTP/1.1" 304 -
127.0.0.1 - - [28/Jul/2021 17:50:38] "GET /precomputed/obj_meshes/mesh/segment_properties/info HTTP/1.1" 304 -
127.0.0.1 - - [28/Jul/2021 17:50:38] "GET /precomputed/obj_meshes/mesh/101:0 HTTP/1.1" 304 -
127.0.0.1 - - [28/Jul/2021 17:50:38] "GET /precomputed/obj_meshes/mesh/100:0 HTTP/1.1" 304 -
127.0.0.1 - - [28/Jul/2021 17:50:38] "GE

# Screenshot of the local neuroglancer instance should be like below..

### The <span style="color:blue">*blue*</span> neuron is <span style="color:blue">*light level mesh (A1)*</span> and the <span style="color:magenta">*magenta*</span> neuron is <span style="color:magenta"> *light level mesh  (A2)*</span> both created from `obj` format 

![alt text](../pics/obj2precomputedmeshes.png "Title")

## Step 3: Close the dataserver..

In [ ]:
closeviewer()
closedataserver()